# TOI-5807.01 End-to-End Validation (Consolidated)

This notebook consolidates the workflow from:

- `04-real-candidate-validation.ipynb`
- `05-extended-metrics.ipynb`
- `06-toi-5807-robustness.ipynb`

…but uses the newer researcher-focused API helpers to reduce notebook glue:

- `load_tutorial_target(...)` (dataset loading)
- `run_candidate_workflow(...)` (thin orchestration)
- `format_vetting_table(...)` / `summarize_bundle(...)` (reporting)
- `export_bundle(...)` (JSON/CSV/Markdown exports)

This notebook is **metrics-first**: it computes evidence and diagnostics but does not impose new validation thresholds beyond the FPP conventions.


In [ ]:
from __future__ import annotations

import tempfile

from pathlib import Path

import numpy as np

import bittr_tess_vetter.api as btv

# Optional: FPP helpers (TRICERATOPS+)
from bittr_tess_vetter.api.fpp import ContrastCurve, calculate_fpp
from bittr_tess_vetter.api.io import PersistentCache

# -----------------------------------------------------------------------------
# Controls (offline-first)
# -----------------------------------------------------------------------------
NETWORK = False  # enables catalog queries (V06/V07) and FPP network dependencies
RUN_FPP = False  # requires NETWORK=True and TRICERATOPS deps
RUN_MULTI_SECTOR_TPFS = False  # requires NETWORK=True and lightkurve

TIC_ID = 188646744
TOI_LABEL = "TOI-5807.01"

# Sky coordinates for TIC 188646744 (used for catalog checks when NETWORK=True)
# Source: SIMBAD / TIC
RA_DEG = 304.12005
DEC_DEG = 11.08344


## 1) Load the tutorial dataset (offline)

This repository includes a small tutorial dataset for TIC 188646744:

- Light curves for sectors 55 / 75 / 82 / 83 (`sector*_pdcsap.csv`)
- A representative TPF stamp for sector 83 (`sector83_tpf.npz`)
- A PHARO AO contrast curve (`PHARO_Kcont_plot.tbl`)

The loader returns a `LocalDataset` with `lc_by_sector` and (optionally) `tpf_by_sector`.


In [ ]:
ds = btv.load_tutorial_target("tic188646744")
ds.summary()

## 2) Candidate ephemeris + stellar parameters

We start with a reasonable ephemeris and stellar parameters (as in the original tutorials).

If you want live ExoFOP numbers, you can add a `NETWORK=True` block to query them, but this notebook defaults to deterministic, offline values.


In [ ]:
# Fallback ephemeris (BTJD = BJD - 2457000)
PERIOD_DAYS = 14.2423724
T0_BTJD = 3540.26317  # ~2460540.26317 BJD - 2457000
DURATION_HOURS = 4.046
DEPTH_PPM = 253.0

# Approx stellar parameters (used by fit + some diagnostics)
stellar = btv.StellarParams(radius=1.65, mass=1.47, teff=6816.0, logg=4.17)

ephem0 = btv.Ephemeris(period_days=PERIOD_DAYS, t0_btjd=T0_BTJD, duration_hours=DURATION_HOURS)
cand0 = btv.Candidate(ephemeris=ephem0, depth_ppm=DEPTH_PPM)

cand0

### Optional refinement: fit a transit model to refine duration / epoch

We stitch the per-sector light curves (per-sector median normalization) and fit a simple transit model. This is mainly for *tutorial reproducibility*.


In [ ]:
stitched = btv.stitch_lightcurves(
    [
        {
            "time": np.asarray(lc.time, dtype=np.float64),
            "flux": np.asarray(lc.flux, dtype=np.float64),
            "flux_err": np.asarray(lc.flux_err, dtype=np.float64),
            "sector": int(sector),
            "quality": np.zeros(len(lc.time), dtype=np.int32),
        }
        for sector, lc in sorted(ds.lc_by_sector.items())
    ]
)

lc_stitched = btv.LightCurve(
    time=stitched.time,
    flux=stitched.flux,
    flux_err=stitched.flux_err,
    quality=stitched.quality,
)

fit = btv.fit_transit(lc_stitched, cand0, stellar)

# `fit_transit` requires optional dependencies (e.g. `batman`).
# If unavailable or the fit fails, fall back to the initial ephemeris.
if getattr(fit, "status", "success") != "success" or float(getattr(fit, "duration_hours", 0.0)) <= 0:
    ephem = ephem0
    candidate = cand0
else:
    ephem = btv.Ephemeris(
        period_days=PERIOD_DAYS,
        t0_btjd=T0_BTJD + float(fit.t0_offset),
        duration_hours=float(fit.duration_hours),
    )
    candidate = btv.Candidate(ephemeris=ephem, depth_ppm=float(fit.transit_depth_ppm))

{
    "t0_btjd": ephem.t0_btjd,
    "duration_hours": ephem.duration_hours,
    "depth_ppm": candidate.depth_ppm,
    "fit_status": getattr(fit, "status", None),
    "fit_error": getattr(fit, "error_message", None),
}

## 3) Baseline vetting (default preset)

We run the default vetting preset on the stitched light curve, plus a per-sector rerun.

Notes:

- This dataset only includes a TPF stamp for sector 83, so pixel checks (V08–V10) will only run for that sector unless you download the missing TPFs.
- Catalog checks (V06/V07) require `NETWORK=True`.


In [ ]:
workflow_default = btv.run_candidate_workflow(
    dataset=ds,
    candidate=candidate,
    stellar=stellar,
    preset="default",
    network=NETWORK,
    ra_deg=RA_DEG,
    dec_deg=DEC_DEG,
    tic_id=TIC_ID,
    run_per_sector=True,
)

print(btv.format_vetting_table(workflow_default.bundle))
workflow_default.per_sector.summary_records if workflow_default.per_sector else None

## 4) Extended metrics (V16–V21)

The `extended` preset adds optional, metrics-only diagnostics. These do not change the semantics of the baseline checks.


In [ ]:
workflow_ext = btv.run_candidate_workflow(
    dataset=ds,
    candidate=candidate,
    stellar=stellar,
    preset="extended",
    network=NETWORK,
    ra_deg=RA_DEG,
    dec_deg=DEC_DEG,
    tic_id=TIC_ID,
    run_per_sector=True,
)

print(btv.format_vetting_table(workflow_ext.bundle))

# Pull out just the extended check metrics for compact inspection
btv.summarize_bundle(
    workflow_ext.bundle,
    check_ids=["V16", "V17", "V18", "V19", "V20", "V21"],
    include_metrics=True,
    include_flags=True,
    include_notes=False,
)

## 5) Robustness: V16 after transit-aware detrending

A common concern is that stellar variability (or stitched offsets) can bias model competition.

We re-run V16 after applying a transit-aware detrend per sector:

- If `wotan` is installed, we use `wotan_flatten(..., return_trend=True)` with a transit mask.
- Otherwise, we fall back to a simple time-windowed median flatten (`flatten`).

This is a diagnostic: we are looking for *stability* of the V16 preference, not imposing a new threshold.


In [ ]:
lc_by_sector_detrended: dict[int, btv.LightCurve] = {}

for sector, lc in sorted(ds.lc_by_sector.items()):
    t = np.asarray(lc.time, dtype=np.float64)
    f = np.asarray(lc.flux, dtype=np.float64)
    e = np.asarray(lc.flux_err, dtype=np.float64)

    in_tr = btv.get_in_transit_mask(t, PERIOD_DAYS, ephem.t0_btjd, ephem.duration_hours)

    if btv.WOTAN_AVAILABLE:
        f_flat, trend = btv.wotan_flatten(
            t,
            f,
            window_length=0.7,  # days; keep > duration
            method="biweight",
            transit_mask=in_tr,
            return_trend=True,
        )
        e_flat = e / trend
    else:
        # No trend returned; keep flux_err unchanged (diagnostic-only)
        f_flat = btv.flatten(t, f, window_length=0.7)
        e_flat = e

    lc_by_sector_detrended[int(sector)] = btv.LightCurve(time=t, flux=f_flat, flux_err=e_flat)

stitched_det = btv.stitch_lightcurves(
    [
        {
            "time": np.asarray(lc.time, dtype=np.float64),
            "flux": np.asarray(lc.flux, dtype=np.float64),
            "flux_err": np.asarray(lc.flux_err, dtype=np.float64),
            "sector": int(sector),
            "quality": np.zeros(len(lc.time), dtype=np.int32),
        }
        for sector, lc in sorted(lc_by_sector_detrended.items())
    ]
)

lc_det = btv.LightCurve(time=stitched_det.time, flux=stitched_det.flux, flux_err=stitched_det.flux_err)

v16_raw = btv.vet_candidate(lc_stitched, candidate, preset="extended", checks=["V16"], network=False)
v16_det = btv.vet_candidate(lc_det, candidate, preset="extended", checks=["V16"], network=False)

def _v16_summary(bundle: btv.VettingBundleResult) -> dict:
    r = bundle.get_result("V16")
    return {
        "winner": None if r is None else r.metrics.get("winner"),
        "label": None if r is None else r.metrics.get("model_competition_label"),
        "winner_margin_bic": None if r is None else r.metrics.get("winner_margin_bic"),
        "artifact_risk": None if r is None else r.metrics.get("artifact_risk"),
    }

{
    "v16_original": _v16_summary(v16_raw),
    "v16_detrended": _v16_summary(v16_det),
}

## 6) Robustness: inspect V19 phase-shift events

V19 reports harmonic plausibility metrics and also detects **phase-shift events** (strong non-transit features at other phases).

If V19 finds multiple significant events, that is a strong prompt to inspect the light curve and sector consistency.


In [ ]:
v19 = btv.vet_candidate(lc_stitched, candidate, preset="extended", checks=["V19"], network=False)

r19 = v19.get_result("V19")
r19.metrics if r19 is not None else None

## 7) Optional: multi-sector pixel vetting (V08–V10)

The bundled dataset includes only a sector 83 TPF stamp. For true multi-sector consistency, download per-sector TPFs and pass them into the per-sector workflow.

This section is **optional** and requires:

- `NETWORK=True`
- `lightkurve` installed
- MAST access


In [ ]:
if RUN_MULTI_SECTOR_TPFS:
    import lightkurve as lk

    tpf_by_sector: dict[int, btv.TPFStamp] = {}

    for sector in sorted(ds.lc_by_sector.keys()):
        print(f"Downloading TPF for sector {sector}...")
        search = lk.search_targetpixelfile(f"TIC {TIC_ID}", sector=int(sector), exptime=120)
        if len(search) == 0:
            print(f"  No TPF found for sector {sector}")
            continue
        tpf = search.download()
        if tpf is None:
            print(f"  Download failed for sector {sector}")
            continue

        # Convert lightkurve TPF -> API TPFStamp
        tpf_by_sector[int(sector)] = btv.TPFStamp(
            time=np.asarray(tpf.time.value, dtype=np.float64),
            flux=np.asarray(tpf.flux.value, dtype=np.float64),
            flux_err=np.asarray(tpf.flux_err.value, dtype=np.float64),
            wcs=tpf.wcs,
            aperture_mask=np.asarray(tpf.pipeline_mask, dtype=bool),
            quality=np.asarray(tpf.quality, dtype=np.int32),
        )

    workflow_pixels = btv.run_candidate_workflow(
        lc_by_sector=ds.lc_by_sector,
        tpf_by_sector=tpf_by_sector,
        candidate=candidate,
        stellar=stellar,
        preset="extended",
        network=NETWORK,
        ra_deg=RA_DEG,
        dec_deg=DEC_DEG,
        tic_id=TIC_ID,
        run_per_sector=True,
    )

    # Print the per-sector pixel check statuses
    for sector, bundle in sorted(workflow_pixels.per_sector.bundles_by_sector.items()):
        r08 = bundle.get_result("V08")
        r09 = bundle.get_result("V09")
        r10 = bundle.get_result("V10")
        print(
            f"sector {sector}: V08={None if r08 is None else r08.status} "
            f"V09={None if r09 is None else r09.status} "
            f"V10={None if r10 is None else r10.status}"
        )
else:
    print("Skipping multi-sector TPF download (set RUN_MULTI_SECTOR_TPFS=True).")

## 8) Optional: AO-assisted FPP (TRICERATOPS+)

This reproduces the AO-assisted statistical validation step using the bundled PHARO contrast curve.

Notes:

- This requires `NETWORK=True` and TRICERATOPS dependencies.
- FPP is outside the vetting checks; it is a separate, well-established statistical validation convention.


In [ ]:
def _parse_exofop_contrast_curve_tbl(path: Path) -> ContrastCurve:
    # ExoFOP-style table: header lines + numeric columns. We keep it simple:
    # parse any lines that start with a number into (sep_arcsec, dmag).
    seps: list[float] = []
    dmags: list[float] = []
    for line in path.read_text().splitlines():
        s = line.strip()
        if not s or s.startswith("#"):
            continue
        parts = s.replace(",", " ").split()
        if len(parts) < 2:
            continue
        try:
            seps.append(float(parts[0]))
            dmags.append(float(parts[1]))
        except Exception:
            continue
    return ContrastCurve(
        separation_arcsec=np.asarray(seps, dtype=np.float64),
        delta_mag=np.asarray(dmags, dtype=np.float64),
        filter="K",
    )


if RUN_FPP:
    if not NETWORK:
        raise ValueError("Set NETWORK=True for FPP")

    # -----------------------------------------------------------------
    # Match tutorial 04's pattern:
    # - Create a temporary PersistentCache directory
    # - Populate it with per-sector LightCurveData entries keyed via make_data_ref
    #   so TRICERATOPS can consume the cached LCs
    # -----------------------------------------------------------------
    cache_dir = Path(tempfile.mkdtemp(prefix="btv_tutorial_fpp_"))
    cache = PersistentCache(cache_dir=cache_dir)

    for sector, lc in sorted(ds.lc_by_sector.items()):
        t = np.asarray(lc.time, dtype=np.float64)
        f = np.asarray(lc.flux, dtype=np.float64)
        e = np.asarray(lc.flux_err, dtype=np.float64)
        q = np.zeros(len(t), dtype=np.int32)
        valid = np.isfinite(t) & np.isfinite(f) & np.isfinite(e)

        lc_data = btv.LightCurveData(
            time=t,
            flux=f,
            flux_err=e,
            quality=q,
            valid_mask=valid,
            tic_id=TIC_ID,
            sector=int(sector),
            cadence_seconds=120.0,
        )
        key = btv.make_data_ref(TIC_ID, int(sector), "pdcsap")
        cache.put(key, lc_data)

    print(f"Cache directory: {cache_dir}")
    cc_path = ds.root / "PHARO_Kcont_plot.tbl"
    cc = _parse_exofop_contrast_curve_tbl(cc_path)

    fpp = calculate_fpp(
        cache=cache,
        tic_id=TIC_ID,
        period=ephem.period_days,
        t0=ephem.t0_btjd,
        depth_ppm=float(candidate.depth_ppm or DEPTH_PPM),
        duration_hours=ephem.duration_hours,
        sectors=sorted(ds.lc_by_sector.keys()),
        stellar_radius=stellar.radius,
        stellar_mass=stellar.mass,
        preset="fast",
        contrast_curve=cc,
    )

    fpp
else:
    print("Skipping FPP (set RUN_FPP=True).")

## 9) Export a shareable report

The export helper can emit JSON / CSV / Markdown without manual formatting.


In [ ]:
md = btv.export_bundle(workflow_ext.bundle, format="md", title=f"{TOI_LABEL} ({TIC_ID})")
md[:1500]